In [1]:
import numpy as np
import pandas as pd
#import math
import warnings
warnings.filterwarnings('ignore')

## Read the macro data from Welch and Goyal (2008)

In [3]:
path_macro = '/Users/jerryzhang/Downloads/PredictorData2023.csv'
macro = pd.read_csv('%s' %path_macro)

print(' dimensions and observations: ', macro.shape)

#drop the comma
#convert to float

macro['Index']= macro['Index'].str.replace(r',', '')
macro['Index']= macro['Index'].astype(float, errors = 'raise')

#Calculate some macro data:
#earnings-price ratio(ep):=log(E12)-log(Index)
#term-spread(tms):=lty-tbl
#default-spread(dfy):=BAA-AAA
#dividen-to-price ratio(dp):=log(D12)-log(Index)

macro['e/p'] = np.log(macro['E12']) - np.log(macro['Index'])
macro['tms'] = macro['lty'] - macro['tbl']
macro['dfy'] = macro['BAA'] - macro['AAA']
macro['dp'] = np.log(macro['D12']) - np.log(macro['Index'])

#macro=macro.rename(columns={'b/m':'bm'})
#can do this later
#print(list(macro))

#drop some unused data
macro.drop(['Index','E12', 'D12', 'AAA', 'BAA','lty', 'Rfree', 'CRSP_SPvwx', 'CRSP_SPvw', 'infl', 'ltr', 'corpr','csp', ], axis=1, inplace=True)
#print(list(macro))

 dimensions and observations:  (1838, 18)


## Read the stocks level data from GKX (2020)

In [5]:
path_stocks = '/Users/jerryzhang/Downloads/datashare/GKX_20201231.csv'
stocksdata = pd.read_csv('%s' %path_stocks)
print('dimensions and observations: ', stocksdata.shape)
covardata = list(stocksdata.columns.values)
#print(covardata)           
#to demonstrate x_{i,t} variables' names
#stocksdata.describe()

dimensions and observations:  (4345508, 101)


## Merge the data, i.e. z_{i,t}=x_{i,t}\otimes c_t

In [7]:
#truncate the time in a proper format

macro['yyyymm'] = pd.to_datetime(macro['yyyymm'], format='%Y%m')
stocksdata['yyyymm'] = stocksdata['DATE'].astype('str').str[:6]
stocksdata['yyyymm'] = pd.to_datetime(stocksdata['yyyymm'], format='%Y%m')

#merge two datasets

stocksdata2 = pd.merge(stocksdata, macro, how="left", on="yyyymm")
stocksdata2.drop(['yyyymm'], axis=1, inplace=True)

##save the data as a new file

In [9]:
stocksdata2.to_csv("merged_dataset.csv", index=None)